In [1]:
# imports
import pandas as pd
import numpy as np
import re
import nltk
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
from nltk.corpus import stopwords

In [2]:
# import configurations
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()
nltk.download('stopwords')
english_stop_words = stopwords.words('english')
add_stopwords = ['bitcoin', 'btc', 'crypto', 'cryptocurrency', 'cryptocurrencies',
                'ethereum', 'eth', 'price', 'prices', 'money', 'dollars']
english_stop_words = english_stop_words + add_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chayisara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# read file
df = pd.read_pickle("../data/interim/tweets_verified_2020-2021.pkl")
df.head()
df.shape
df = df.drop_duplicates(subset='tweet', keep=False)
df.shape
df[(df['id'] == 1212277383287328769) | (df['id'] == 1212277146401402880)]

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to
0,2020-01-01 00:08:28-05:00,1212239143687741440,1212239143687741440,20646945,dumbfoundead,dumbfoundead.eth,Bitcoin the worst decision i made this decade,en,[],[],...,9,15,409,[],[],https://twitter.com/dumbfoundead/status/121223...,<NA>,False,<NA>,[]
1,2020-01-01 00:54:35-05:00,1212250748815208448,1212250748815208448,636023721,neuroecology,Adam J Calhoun,If you ask someone what they'd do if they went...,en,[],[],...,6,0,4,[],[],https://twitter.com/neuroecology/status/121225...,<NA>,False,<NA>,[]
2,2020-01-01 02:00:25-05:00,1212267316789952512,1212267316789952512,631810714,cnbctv18news,CNBC-TV18,#bitcoin rally begun in 2013 and it reached a ...,en,[],['https://www.cnbctv18.com/market/currency/bit...,...,0,0,4,['bitcoin'],[],https://twitter.com/CNBCTV18News/status/121226...,<NA>,False,<NA>,[]
3,2020-01-01 02:18:48-05:00,1212271940502638593,1212271940502638593,14654494,excellion,Samson Mow,Wishing all #Bitcoin Ultra Enthusiasts a very ...,en,"[{'screen_name': 'blockstream', 'name': 'block...",[],...,10,17,158,"['bitcoin', 'liquidnetwork']",[],https://twitter.com/Excellion/status/121227194...,<NA>,True,https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg,[]
4,2020-01-01 02:39:29-05:00,1212277146401402880,1212277146401402880,1066972567943053312,hindustantimes,Hindustan Times,"Breaking down Bitcoin’s 9,000,000% rise in las...",en,[],['http://www.hindustantimes.com/tech/breaking-...,...,0,0,0,[],[],https://twitter.com/HindustanTimes/status/1212...,<NA>,False,<NA>,[]


(312521, 21)

(309072, 21)

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to
4,2020-01-01 02:39:29-05:00,1212277146401402880,1212277146401402880,1066972567943053312,hindustantimes,Hindustan Times,"Breaking down Bitcoin’s 9,000,000% rise in las...",en,[],['http://www.hindustantimes.com/tech/breaking-...,...,0,0,0,[],[],https://twitter.com/HindustanTimes/status/1212...,<NA>,False,<NA>,[]
5,2020-01-01 02:40:25-05:00,1212277383287328769,1212277383287328769,3305925577,httech,Hindustan Times Tech,"Breaking down Bitcoin’s 9,000,000% rise in las...",en,"[{'screen_name': 'httweets', 'name': 'hindusta...",['https://www.hindustantimes.com/tech/breaking...,...,0,0,0,[],[],https://twitter.com/HTTech/status/121227738328...,<NA>,False,<NA>,[]


In [67]:
def repl(matchObj):
    char = matchObj.group(1)
    return "%s%s" % (char, char)

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

def clean_tweet_1(tweet):
    # remove URL "https://t.co/"
    new_url = re.sub(r"https://t.co/[A-Za-z0-9]+", " ", tweet)
    
    # remove mention
    new_mention = re.sub(r"@[A-Za-z0-9!#%&*;_\$\.]+", " ", new_url)
    
    # remove # ,turning hashtags to the typical words.
    new_symbol = re.sub(r"\W+", " ", new_mention)
    
    # remove number
    new_number = re.sub(r"[0-9]", "", new_symbol)
    
    return new_number

def clean_tweet_2(tweet):
    # reduce character sequences >3 to 3
    new_sequence = re.sub(re.compile(r"(\w)\1+"), repl, tweet)

    # remove 2-character words
    new_twochar = re.sub(r"\b[a-z]{1,2}\b", " ", new_sequence)
    
    # replace 2 or more spaces with a single space.
    new_space = re.sub(r"\s+", " ", new_twochar)
    
    # remove spaces front and back
    new_space_end = re.sub(r"^\s+|\s$", "", new_space)
    
    return new_space_end

def clean_tweet_money(tweet):

    new_money = re.sub(r"([0-9]+) ?(k)", r"\1000", tweet)
    remove_money = re.sub(r"([0-9]+)(,?)([0-9]+)(,?)([0-9]+?)", r"\1\3\5", new_money)
    all_money = re.findall(r"([0-9]+)", remove_money)
    
    return all_money



In [68]:
# tweet = '3 k'
# tweets = '9,000'
# new_money = re.sub(r"([0-9]+) ?(k)", r"\1000", tweet)
# remove_money = re.sub(r"([0-9]+)(,?)([0-9]+)(,?)([0-9]+?)", r"\1\3\5", tweets)
# new_money
# remove_money


In [69]:
# language EN only
df = df[df['language'] == 'en']
df['tweet_clean'] = pd.DataFrame(df['tweet'].str.lower())

In [70]:
# apply money extract
all_money = df['tweet_clean'].apply(clean_tweet_money)
all_money

0                                 []
1                                 []
2             [2013, 20000, 2017, 3]
3              [2020, 1, 2, 2, 8, 0]
4                    [9000000, 9478]
                     ...            
308750                            []
308751                            []
308752                            []
308753    [2022, 76, 4, 1, 76, 4, 1]
308754                            []
Name: tweet_clean, Length: 308755, dtype: object

In [73]:
# apply cleaning
df['tweet_clean'] = df['tweet_clean'].progress_apply(clean_tweet_1)
df['tweet_clean'] = remove_stop_words(df['tweet_clean'])
df['tweet_clean'] = df['tweet_clean'].progress_apply(clean_tweet_2)
df['tweet_clean']

  0%|          | 0/308755 [00:00<?, ?it/s]

  0%|          | 0/308755 [00:00<?, ?it/s]

0                                worst decision made decade
1         ask someone went back ten years use informatio...
2                                  rally begun reached peak
3         wishing ultra enthusiasts happy new year much ...
4            breaking rise last decade left skeptics aghast
                                ...                        
308750                                          fair enough
308751                                       happy new year
308752             buy amp chill defund political parasites
308753                          faces uncertain record year
308754                 great example fellow familia brother
Name: tweet_clean, Length: 308755, dtype: object

In [74]:
df.head()
df[['tweet_clean', 'tweet']]

,created_at,id,conversation_id,user_id,username,name,tweet,language,mentions,urls,...,retweets_count,likes_count,hashtags,cashtags,link,quote_url,video,thumbnail,reply_to,tweet_clean
0,2020-01-01 00:08:28-05:00,1212239143687741440,1212239143687741440,20646945,dumbfoundead,dumbfoundead.eth,Bitcoin the worst decision i made this decade,en,[],[],...,15,409,[],[],https://twitter.com/dumbfoundead/status/121223...,<NA>,False,<NA>,[],worst decision made decade
1,2020-01-01 00:54:35-05:00,1212250748815208448,1212250748815208448,636023721,neuroecology,Adam J Calhoun,If you ask someone what they'd do if they went...,en,[],[],...,0,4,[],[],https://twitter.com/neuroecology/status/121225...,<NA>,False,<NA>,[],ask someone went back ten years use informatio...
2,2020-01-01 02:00:25-05:00,1212267316789952512,1212267316789952512,631810714,cnbctv18news,CNBC-TV18,#bitcoin rally begun in 2013 and it reached a ...,en,[],['https://www.cnbctv18.com/market/currency/bit...,...,0,4,['bitcoin'],[],https://twitter.com/CNBCTV18News/status/121226...,<NA>,False,<NA>,[],rally begun reached peak
3,2020-01-01 02:18:48-05:00,1212271940502638593,1212271940502638593,14654494,excellion,Samson Mow,Wishing all #Bitcoin Ultra Enthusiasts a very ...,en,"[{'screen_name': 'blockstream', 'name': 'block...",[],...,17,158,"['bitcoin', 'liquidnetwork']",[],https://twitter.com/Excellion/status/121227194...,<NA>,True,https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg,[],wishing ultra enthusiasts happy new year much ...
4,2020-01-01 02:39:29-05:00,1212277146401402880,1212277146401402880,1066972567943053312,hindustantimes,Hindustan Times,"Breaking down Bitcoin’s 9,000,000% rise in las...",en,[],['http://www.hindustantimes.com/tech/breaking-...,...,0,0,[],[],https://twitter.com/HindustanTimes/status/1212...,<NA>,False,<NA>,[],breaking rise last decade left skeptics aghast


,tweet_clean,tweet
0,worst decision made decade,Bitcoin the worst decision i made this decade
1,ask someone went back ten years use informatio...,If you ask someone what they'd do if they went...
2,rally begun reached peak,#bitcoin rally begun in 2013 and it reached a ...
3,wishing ultra enthusiasts happy new year much ...,Wishing all #Bitcoin Ultra Enthusiasts a very ...
4,breaking rise last decade left skeptics aghast,"Breaking down Bitcoin’s 9,000,000% rise in las..."
...,...,...
308750,fair enough,@Bitcoin__art Fair enough 😅😅😅
308751,happy new year,Happy New Year #Bitcoin https://t.co/HdUQNDKUUA
308752,buy amp chill defund political parasites,"@rubiconcapital_ Buy bitcoin &amp; chill, defu..."
308753,faces uncertain record year,Bitcoin faces uncertain 2022 after record year...


In [75]:
# export as pickle
df = df.astype({'tweet':'string'}).reset_index(drop=True)
df.to_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

In [76]:
# try reading pickle
# df_test = pd.read_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

# display(df_test.dtypes)
# display(df_test)

In [77]:
# export as pickle
all_money.to_pickle("../data/cleaned/all_money.pkl")